# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 20 2022 9:13AM,239916,16,SAR00000344,Sartorius Stedim North America,Released
1,Jun 20 2022 9:07AM,239913,10,0085778670,ISDIN Corporation,Released
2,Jun 20 2022 9:07AM,239913,10,0085778673,ISDIN Corporation,Released
3,Jun 20 2022 9:07AM,239913,10,0085778688,ISDIN Corporation,Released
4,Jun 20 2022 9:07AM,239913,10,0085778695,ISDIN Corporation,Released
5,Jun 20 2022 9:07AM,239913,10,0085778689,ISDIN Corporation,Released
6,Jun 20 2022 9:07AM,239913,10,0085778709,ISDIN Corporation,Released
7,Jun 20 2022 9:07AM,239913,10,0085778713,ISDIN Corporation,Released
8,Jun 20 2022 9:07AM,239913,10,0085778716,ISDIN Corporation,Released
9,Jun 20 2022 9:07AM,239913,10,0085778714,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,239909,Released,8
19,239912,Released,1
20,239913,Released,54
21,239914,Released,1
22,239916,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
239909,NaN,8.0
239912,NaN,1.0
239913,NaN,54.0
239914,NaN,1.0
239916,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
239878,0.0,1.0
239891,1.0,0.0
239892,0.0,1.0
239894,24.0,9.0
239896,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
239878,0,1
239891,1,0
239892,0,1
239894,24,9
239896,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,239878,0,1
1,239891,1,0
2,239892,0,1
3,239894,24,9
4,239896,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,239878,,1
1,239891,1,
2,239892,,1
3,239894,24,9
4,239896,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 20 2022 9:13AM,239916,16,Sartorius Stedim North America
1,Jun 20 2022 9:07AM,239913,10,ISDIN Corporation
55,Jun 20 2022 9:04AM,239907,10,ISDIN Corporation
116,Jun 20 2022 9:04AM,239914,10,"Senseonics, Incorporated"
117,Jun 20 2022 9:02AM,239912,10,Bio-PRF
118,Jun 20 2022 9:00AM,239909,10,"Snap Medical Industries, LLC"
126,Jun 20 2022 8:58AM,239903,10,ISDIN Corporation
180,Jun 20 2022 8:55AM,239906,10,Beach Products Inc
192,Jun 20 2022 8:51AM,239905,12,Hush Hush
225,Jun 20 2022 8:46AM,239904,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 20 2022 9:13AM,239916,16,Sartorius Stedim North America,,1
1,Jun 20 2022 9:07AM,239913,10,ISDIN Corporation,,54
2,Jun 20 2022 9:04AM,239907,10,ISDIN Corporation,,61
3,Jun 20 2022 9:04AM,239914,10,"Senseonics, Incorporated",,1
4,Jun 20 2022 9:02AM,239912,10,Bio-PRF,,1
5,Jun 20 2022 9:00AM,239909,10,"Snap Medical Industries, LLC",,8
6,Jun 20 2022 8:58AM,239903,10,ISDIN Corporation,,54
7,Jun 20 2022 8:55AM,239906,10,Beach Products Inc,,12
8,Jun 20 2022 8:51AM,239905,12,Hush Hush,1,32
9,Jun 20 2022 8:46AM,239904,10,Emerginnova,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 9:13AM,239916,16,Sartorius Stedim North America,1,
1,Jun 20 2022 9:07AM,239913,10,ISDIN Corporation,54,
2,Jun 20 2022 9:04AM,239907,10,ISDIN Corporation,61,
3,Jun 20 2022 9:04AM,239914,10,"Senseonics, Incorporated",1,
4,Jun 20 2022 9:02AM,239912,10,Bio-PRF,1,
5,Jun 20 2022 9:00AM,239909,10,"Snap Medical Industries, LLC",8,
6,Jun 20 2022 8:58AM,239903,10,ISDIN Corporation,54,
7,Jun 20 2022 8:55AM,239906,10,Beach Products Inc,12,
8,Jun 20 2022 8:51AM,239905,12,Hush Hush,32,1
9,Jun 20 2022 8:46AM,239904,10,Emerginnova,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 9:13AM,239916,16,Sartorius Stedim North America,1,
1,Jun 20 2022 9:07AM,239913,10,ISDIN Corporation,54,
2,Jun 20 2022 9:04AM,239907,10,ISDIN Corporation,61,
3,Jun 20 2022 9:04AM,239914,10,"Senseonics, Incorporated",1,
4,Jun 20 2022 9:02AM,239912,10,Bio-PRF,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 9:13AM,239916,16,Sartorius Stedim North America,1.0,NaN
1,Jun 20 2022 9:07AM,239913,10,ISDIN Corporation,54.0,NaN
2,Jun 20 2022 9:04AM,239907,10,ISDIN Corporation,61.0,NaN
3,Jun 20 2022 9:04AM,239914,10,"Senseonics, Incorporated",1.0,NaN
4,Jun 20 2022 9:02AM,239912,10,Bio-PRF,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 20 2022 9:13AM,239916,16,Sartorius Stedim North America,1.0,0.0
1,Jun 20 2022 9:07AM,239913,10,ISDIN Corporation,54.0,0.0
2,Jun 20 2022 9:04AM,239907,10,ISDIN Corporation,61.0,0.0
3,Jun 20 2022 9:04AM,239914,10,"Senseonics, Incorporated",1.0,0.0
4,Jun 20 2022 9:02AM,239912,10,Bio-PRF,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2638929,194.0,1.0
12,239905,32.0,1.0
16,1199512,5.0,0.0
18,239898,1.0,0.0
19,479794,9.0,25.0
20,239899,15.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2638929,194.0,1.0
1,12,239905,32.0,1.0
2,16,1199512,5.0,0.0
3,18,239898,1.0,0.0
4,19,479794,9.0,25.0
5,20,239899,15.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,194.0,1.0
1,12,32.0,1.0
2,16,5.0,0.0
3,18,1.0,0.0
4,19,9.0,25.0
5,20,15.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,194.0
1,12,Released,32.0
2,16,Released,5.0
3,18,Released,1.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Executing,1.0,1.0,0.0,0.0,25.0,0.0
Released,194.0,32.0,5.0,1.0,9.0,15.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Executing,1.0,1.0,0.0,0.0,25.0,0.0
1,Released,194.0,32.0,5.0,1.0,9.0,15.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Executing,1.0,1.0,0.0,0.0,25.0,0.0
1,Released,194.0,32.0,5.0,1.0,9.0,15.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()